In [49]:
import math
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

In [87]:
def preprocess_features(df):
    #normalized_df=(df-df.mean())/df.std()
    normalized_df=(df-df.min())/(df.max()-df.min())
#     processed_features = pd.DataFrame()
#     processed_features["Act_A"] = (df["Act_A"] / 100.0)
#     processed_features["Act_B"] = (df["Act_B"] / 150.0)
#     processed_features["Frac_A"] = (df["Frac_A"] / 100.0)
#     processed_features["Gas_A"] = (df["Act_A"] / 1000.0)
#     processed_features["Gas_B"] = (df["Act_A"] / 1000.0)
#     processed_features["Act_A"] = (df["Act_A"] / 1000.0)
    print df.mean()
    return normalized_df

In [88]:
features = pd.read_pickle('phaseSeparationFeatures.pkl')

targets = pd.DataFrame({'Phase_Sep': features['Phase_Sep']})

features = features.drop(columns=['Phase_Sep', 'Timestep'])

training_examples = preprocess_features(features.head(int(1080*.7)))
training_targets = targets.head(int(1080*.7))

validation_examples = preprocess_features(features.tail(int(1080*.3)))
validation_targets = targets.tail(int(1080*.3))

Act_A        3.690476e+01
Act_B        9.892857e+01
Frac_A       5.000000e+01
Gas_A        4.073569e+07
Gas_B        4.995230e+03
Gas_tot      3.202731e+03
Dense_A      8.197962e+03
Dense_B      2.504770e+03
Dense_tot    4.297269e+03
Lg_clust     6.802038e+03
dtype: float64
Act_A        6.944444e+01
Act_B        1.136111e+02
Frac_A       5.000000e+01
Gas_A        4.656632e+07
Gas_B        1.892503e+03
Gas_tot      1.225840e+03
Dense_A      3.118343e+03
Dense_B      5.607497e+03
Dense_tot    6.274160e+03
Lg_clust     1.188166e+04
dtype: float64


In [89]:
# Double-check that we've done the right thing.
print("Training examples summary:")
display.display(training_examples.describe())
print("Validation examples summary:")
display.display(validation_examples.describe())

print("Training targets summary:")
display.display(training_targets.describe())
print("Validation targets summary:")
display.display(validation_targets.describe())

Training examples summary:


,Act_A,Act_B,Frac_A,Gas_A,Gas_B,Gas_tot,Dense_A,Dense_B,Dense_tot,Lg_clust
count,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000
mean,0.263605,0.635204,0.500000,0.651775,0.363082,0.229880,0.503138,0.205950,0.349656,0.496862
std,0.276380,0.279888,0.322962,0.261262,0.318352,0.197943,0.376846,0.228175,0.322112,0.376846
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.071429,0.428571,0.250000,0.462583,0.092545,0.102535,0.149799,0.000000,0.000000,0.000000
50%,0.178571,0.714286,0.500000,0.700001,0.240882,0.171565,0.422389,0.149153,0.293653,0.577611
75%,0.285714,0.857143,0.750000,0.866668,0.605126,0.297809,1.000000,0.300095,0.631814,0.850201
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Validation examples summary:


,Act_A,Act_B,Frac_A,Gas_A,Gas_B,Gas_tot,Dense_A,Dense_B,Dense_tot,Lg_clust
count,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000
mean,0.486111,0.595679,0.500000,0.739597,0.222052,0.281216,0.233608,0.449078,0.476800,0.766392
std,0.333559,0.289897,0.323248,0.193513,0.171053,0.182626,0.161408,0.281217,0.297326,0.161408
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.250000,0.333333,0.250000,0.589892,0.090648,0.140896,0.118240,0.226319,0.235178,0.675607
50%,0.500000,0.611111,0.500000,0.727616,0.194771,0.261397,0.191830,0.450346,0.483922,0.808170
75%,0.750000,0.888889,0.750000,0.931907,0.317336,0.375633,0.324393,0.680665,0.729165,0.881760
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Training targets summary:


,Phase_Sep
count,756.000000
mean,0.616402
std,0.486584
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


Validation targets summary:


,Phase_Sep
count,324.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [90]:
def construct_feature_columns(input_features):
    """Construct the TensorFlow Feature Columns.

    Args:
      input_features: The names of the numerical input features to use.
    Returns:
    A set of feature columns
    """
    return set([tf.feature_column.numeric_column(my_feature)
              for my_feature in input_features])

def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
    
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
        ds = ds.shuffle(10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [91]:
def train_model(
    learning_rate,
    steps,
    batch_size,
    feature_columns,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
    """Trains a linear regression model.
  
    In addition to training, this function also prints training progress information,
    as well as a plot of the training and validation loss over time.
  
    Args:
      learning_rate: A `float`, the learning rate.
      steps: A non-zero `int`, the total number of training steps. A training step
        consists of a forward and backward pass using a single batch.
      feature_columns: A `set` specifying the input feature columns to use.
      training_examples: A `DataFrame` containing one or more columns from
       `california_housing_dataframe` to use as input features for training.
      training_targets: A `DataFrame` containing exactly one column from
      `california_housing_dataframe` to use as target for training.
      validation_examples: A `DataFrame` containing one or more columns from
      `california_housing_dataframe` to use as input features for validation.
     validation_targets: A `DataFrame` containing exactly one column from
       `california_housing_dataframe` to use as target for validation.
      
    Returns:
    A `LinearRegressor` object trained on the training data.
    """

    periods = 10
    steps_per_period = steps / periods

    # Create a linear regressor object.
    my_optimizer = tf.train.FtrlOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
    linear_regressor = tf.estimator.LinearRegressor(
        feature_columns=feature_columns,
        optimizer=my_optimizer
    )
  
    training_input_fn = lambda: my_input_fn(training_examples, 
                                            training_targets["Phase_Sep"], 
                                            batch_size=batch_size)
    predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                    training_targets["Phase_Sep"], 
                                                    num_epochs=1, 
                                                    shuffle=False)
    predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                                      validation_targets["Phase_Sep"], 
                                                      num_epochs=1, 
                                                      shuffle=False)

    # Train the model, but do so inside a loop so that we can periodically assess
    # loss metrics.
    print("Training model...")
    print("RMSE (on training data):")
    training_rmse = []
    validation_rmse = []
    for period in range (0, periods):
        # Train the model, starting from the prior state.
        linear_regressor.train(
            input_fn=training_input_fn,
            steps=steps_per_period
        )
        # Take a break and compute predictions.
        training_predictions = linear_regressor.predict(input_fn=predict_training_input_fn)
        training_predictions = np.array([item['predictions'][0] for item in training_predictions])
        validation_predictions = linear_regressor.predict(input_fn=predict_validation_input_fn)
        validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])

        # Compute training and validation loss.
        training_root_mean_squared_error = math.sqrt(
            metrics.mean_squared_error(training_predictions, training_targets))
        validation_root_mean_squared_error = math.sqrt(
            metrics.mean_squared_error(validation_predictions, validation_targets))
        # Occasionally print the current loss.
        print("  period %02d : %0.2f" % (period, training_root_mean_squared_error))
        # Add the loss metrics from this period to our list.
        training_rmse.append(training_root_mean_squared_error)
        validation_rmse.append(validation_root_mean_squared_error)
    print("Model training finished.")

  
    # Output a graph of loss metrics over periods.
    plt.ylabel("RMSE")
    plt.xlabel("Periods")
    plt.title("Root Mean Squared Error vs. Periods")
    plt.tight_layout()
    plt.plot(training_rmse, label="training")
    plt.plot(validation_rmse, label="validation")
    plt.legend()

    return linear_regressor

In [ ]:
_ = train_model(
    learning_rate=1.0,
    steps=1000,
    batch_size=100,
    feature_columns=construct_feature_columns(training_examples),
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c3bee4910>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/pl/jp12pcrd4zdbm6thjcshst540000gn/T/tmpayUlKF', '_train_distribute': None, '_save_summary_steps': 100}
Training model...
RMSE (on training data):
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/pl/jp12pcrd4zdbm6thjcshst540000gn/T/tmpayUlKF/model.ckpt-600
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
  period 05 : 0.21
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/pl/jp12pcrd4zdbm6thjcshst540000gn/T/tmpayUlKF/model.ckpt-600
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 600 into /var/folders/pl/jp12pcrd4zdbm6thjcshst540000gn/T/tmpayUlKF/model.ckpt.
INFO:tensorflow:loss = 7.28078, step = 601
INFO:tensorflow:Saving checkpoints for 700 into /var/folders/pl/jp12pcrd4z